In [2]:
#import libraries
import torch
import torch.nn as nn
import shutil
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset,DataLoader,random_split
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
from PIL import Image,ImageFile
import torch.optim as optim
import pandas as pd
import copy
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
import cv2
from torchvision import transforms
import albumentations as augment
from albumentations.pytorch import ToTensorV2
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
from captum.attr import IntegratedGradients
import timm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

2024-11-20 00:35:07.039057: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-20 00:35:07.055835: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/oem/miniconda3/envs/python3.10/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


cuda


In [3]:
seed_val = 420
torch.manual_seed(seed_val)

In [4]:
base_data_dir = '../data/artifact'
fake_dir = os.path.join(base_data_dir, 'generated')  # Fake artworks directory
real_dir = os.path.join(base_data_dir, 'real')  # Real artworks directory

ARTWORK DATASET

In [5]:
#Class to manage artworks with respect to their authenticity

class ArtworkDataset(Dataset):
  def __init__(self,links,transform):
      self.data = links
      self.transform = transform

  def __len__(self):
    return self.data.index.shape[0]
    
  def __getitem__(self,idx):
        img = Image.open(self.data.iloc[idx,0])
        label_index = self.data.iloc[idx, 1]
        if (img.mode != 'RGB'):
            img = img.convert('RGB')

        if self.transform:
          img = self.transform(img)
        return img, label_index


In [6]:
# Create a CSV file with paths to artwork images and their labels (real or fake)
data = [] 

# Iterate over the fake artworks and add their paths and labels to the list
for dirpath, dirnames, filenames in os.walk(fake_dir):
    for filename in filenames:
        if filename.endswith(".jpg"): # only consider jpg files
            filepath = os.path.join(dirpath, filename)
            data.append((filepath, "0"))


# Iterate over the real artworks and add their paths and labels to the list
for dirpath, dirnames, filenames in os.walk(real_dir):
    for filename in filenames:
        if filename.endswith(".jpg"):
            filepath = os.path.join(dirpath, filename)
            data.append((filepath, "1"))  # Label 1 for real artworks

# Convert the list "data" to a pandas dataframe
df = pd.DataFrame(data, columns=["path", "label"])

# Save the dataframe to a CSV file
csv_output_path = os.path.join(base_data_dir, "image_labels.csv")
df.to_csv(csv_output_path, index=False)
print(f"CSV file saved at {csv_output_path}")


CSV file saved at ../data/artifact/image_labels.csv


LOAD PRETRAINED MODEL

In [7]:
model = timm.create_model('convnext_base',pretrained=True, num_classes=2)

model = model.to(device)
print(model)

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=128, out_features=512, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (fc2): Linear(in_features=512, out_features=128, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (norm): LayerNorm((128,), eps=1e-06, elemen

In [8]:
#Setting the model weights to non-trainable
for param in model.parameters(): 
    param.requires_grad = False

In [9]:
#Make the last layer of the model trainable
for p in model.head.parameters(): #instead of fc, we use head
    p.requires_grad=True


SPLIT IN TRAINING AND VALIDATION SET

In [10]:
dataset = df
dataset['label'] = dataset['label'].astype(int)
dataset

,path,label
0,../data/artifact/generated/stylegan3-t-metface...,0
1,../data/artifact/generated/stylegan3-t-metface...,0
2,../data/artifact/generated/stylegan3-t-metface...,0
3,../data/artifact/generated/stylegan3-t-metface...,0
4,../data/artifact/generated/stylegan3-t-metface...,0
...,...,...
88146,../data/artifact/real/paul-albert-besnard_robe...,1
88147,../data/artifact/real/nikolai-ge_christ-and-th...,1
88148,../data/artifact/real/paul-bril_view-of-a-port...,1
88149,../data/artifact/real/felix-vallotton_the-port...,1


In [11]:
#train, validation = train_test_split(dataset.values, stratify=dataset.values[:, 1], test_size=.3, random_state = 1) 

# Split the dataset into two parts (train + validation, and test)
train_val_data, test = train_test_split(dataset.values, test_size=0.1, random_state=seed_val)

# Split the train + validation part into training and validation sets
train, validation = train_test_split(train_val_data, test_size=0.1, random_state=seed_val)

In [12]:
print(f"Test set size: {len(validation)}")

Test set size: 7934


In [13]:
train_links = pd.DataFrame(train, columns = dataset.columns)
validation_links = pd.DataFrame(validation, columns = dataset.columns)
test_links = pd.DataFrame(test, columns = dataset.columns)

In [14]:
train_links[:5]

,path,label
0,../data/artifact/real/felipe-de-vicente_art-67...,1
1,../data/artifact/generated/stylegan3-r-metface...,0
2,../data/artifact/real/johannes-itten_der-bachs...,1
3,../data/artifact/generated/stylegan3-t-metface...,0
4,../data/artifact/real/stanley-spencer_kit-insp...,1


#### DATA LOADERS

In [15]:
# old
data_transforms = transforms.Compose([
                                transforms.Resize(224),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# new
train_transforms = transforms.Compose([
    transforms.Resize(256),                    # Resize to allow cropping
    transforms.RandomCrop(224),                # Crop to 224x224
    transforms.RandomHorizontalFlip(p=0.5),    # Random horizontal flip -- half of the images are mirrored; prevent the model from learning exact position-based cues.
    transforms.RandomRotation(10),             # Small rotation for variability -- helps the model generalize across minor rotations, as real-world artworks and digital images might not always be perfectly aligned.
    transforms.ColorJitter(brightness=0.1, contrast=0.1), # Minimal brightness/contrast -- A brightness and contrast jitter of ±10% introduces slight lighting variability without drastically changing color tones. This adjustment accounts for small lighting and contrast differences that can naturally occur in photographed or scanned artworks, which could otherwise become confounding factors.
    transforms.ToTensor(),                     # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Standard normalization
])

# Validation/test transformations (no augmentation)
val_test_transforms = transforms.Compose([
    transforms.Resize(224),                    # Resize directly to 224x224
    transforms.CenterCrop(224),                # Center crop to ensure consistency
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image
])

batch_size = 32

train_set = ArtworkDataset(train_links, train_transforms)

validation_set = ArtworkDataset(validation_links, val_test_transforms)

test_set = ArtworkDataset(test_links, val_test_transforms)


train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, 
                               drop_last=False,num_workers=6, pin_memory = True)

validation_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=False, 
                               drop_last=False,num_workers=6, pin_memory = True)

test_loader = DataLoader(test_set,batch_size=batch_size, shuffle = False,
                              drop_last=False,num_workers=6, pin_memory = True)

**Early Stopping**

In [16]:
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=5, min_delta=0.001):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.wait = 0
        self.best_loss = None
        self.early_stop = False
                
    def __call__(self, current_loss):
        if self.best_loss is None or (current_loss - self.best_loss) < -self.min_delta:
            self.best_loss = current_loss
            self.wait = 0
        else:
            self.wait += 1
            print(f"INFO: Early stopping counter {self.wait} of {self.patience}")
            if self.wait >= self.patience:
                self.early_stop = True

**Functions for visualisation and logging**

In [17]:
# functions for logging the misclassified images in tensorboard
def denormalize_image(tensor, mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)):
    """Denormalizes a tensor image by applying the inverse of the normalization transform."""
    # Reshape mean and std to match the (C, H, W) shape of the tensor
    mean = torch.tensor(mean).view(3, 1, 1).to(tensor.device)  # Match device as well
    std = torch.tensor(std).view(3, 1, 1).to(tensor.device)
    denormalized = tensor * std + mean
    denormalized = denormalized.clamp(0, 1) # convert to [H, W, C] for matplotlib visualisation

    return denormalized.permute(1, 2, 0).numpy()

def add_labels_to_image(image, true_label, pred_label):
    """Add true and predicted labels to the image."""
    # Convert the image to uint8 if it's in float format
    if image.dtype != np.uint8:
        image = (image * 255).astype(np.uint8)

    # Convert from RGB to BGR for OpenCV
    bgr_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Prepare text without brackets
    text = f'True: {int(true_label)}, Pred: {int(pred_label)}'
    cv2.putText(bgr_image, text, (5, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # Convert back to RGB before returning
    return cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)


In [18]:
def log_misclassified_images(misclassified_images, misclassified_preds, misclassified_true, epoch, writer):
    print("Logging misclassified images...")
    print(len(misclassified_images))
    labeled_images = []

    for i, img in enumerate(misclassified_images):
        img_np = denormalize_image(img)  # Denormalize the image      

        # Ensure the image is in the right format for TensorBoard
        if img_np.shape[-1] == 1:  # Check if the image is grayscale
            img_np = img_np.squeeze(axis=2)  # Remove the channel dimension if it is 1
        elif img_np.shape[0] == 1:  # If the shape is (1, H, W)
            img_np = img_np.squeeze(axis=0)  # Remove the batch dimension
        img_np = np.clip(img_np, 0, 1)  # Ensure the values are between 0 and 1
        img_np = (img_np * 255).astype(np.uint8)     # Scale to 0-255

        true_label = misclassified_true[i]
        pred_label = misclassified_preds[i]

        labeled_image = add_labels_to_image(img_np, true_label, pred_label)
        # labeled_image = labeled_image.transpose(2, 0, 1)
        labeled_images.append(labeled_image)

    # Log image to TensorBoard
    writer.add_images(
        f'Misclassified_Epoch_{epoch}',
        np.array(labeled_images),
        global_step=epoch,
        dataformats='NHWC'
    )


def log_confusion_matrix(all_labels, all_preds, epoch, log_name, writer):
    """Log confusion matrix to TensorBoard."""
    cm = confusion_matrix(all_labels, all_preds)

    # Plot confusion matrix using Seaborn
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Fake', 'True'], yticklabels=['Fake', 'True'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix - Epoch {epoch}')
    plt.tight_layout()

    # Save to buffer and log
    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    image = np.array(Image.open(buf))
    writer.add_image(f'Confusion_Matrix', image, global_step=epoch, dataformats='HWC')
    buf.close()
    plt.close()

##### GRADCAM and Integrated Gradients

In [19]:
print(model.stages[-1].blocks[-1])

ConvNeXtBlock(
  (conv_dw): Conv2d(1024, 1024, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=1024)
  (norm): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
  (mlp): Mlp(
    (fc1): Linear(in_features=1024, out_features=4096, bias=True)
    (act): GELU()
    (drop1): Dropout(p=0.0, inplace=False)
    (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    (drop2): Dropout(p=0.0, inplace=False)
  )
  (drop_path): Identity()
)


In [20]:
def save_grads_hook(module, input, output):
    if output.grad is not None:
        print("Gradients Shape:", output.grad.shape)  # Make sure gradients are available
    else:
        print("No gradients found.")

def apply_grad_cam_old(model, images, preds, true_labels, writer, log_name):
    """Applies Grad-CAM++ to misclassified images and logs them to TensorBoard."""
    target_layer = model.stages[-1].blocks[-1].conv_dw  # target layer
    print("Using target layer:", target_layer)


    cam = GradCAMPlusPlus(model=model, target_layers=[target_layer])

    for idx, (image, true, pred) in enumerate(zip(images, true_labels, preds)):
        # Denormalize the image to [0, 1] range
        image_np = denormalize_image(image, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

        if len(image.shape) == 3:
            image = image.unsqueeze(0)  # Add batch dimension

        image = image.to(device)

        # Ensure gradients are computed
        with torch.set_grad_enabled(True):
            output = model(image)  # Forward pass
            output_class = output[0, predicted_class]  # Get the predicted class score
            output_class.backward()
            output.backward()  # Backpropagate

        hook = target_layer.register_forward_hook(save_grads_hook)
        

        # Generate Grad-CAM visualization
        grayscale_cam = cam(input_tensor=image)

        plt.figure(figsize=(6, 6))
        plt.imshow(grayscale_cam[0], cmap='gray')  # Show in grayscale to see raw activations
        plt.title(f"Raw Grad-CAM Map {idx}")
        plt.colorbar()
        plt.axis('off')
        plt.show()

        # Apply the proper normalization and scaling factor
        grayscale_cam = np.maximum(grayscale_cam[0], 0)  # Clip negative values
        cam_image = np.clip(grayscale_cam * 2, 0, 1)  # Enhance contrast by scaling (tune this value if needed)
        cam_image = plt.get_cmap('jet')(cam_image)  # Apply 'jet' colormap for visualization
        cam_image = cam_image[..., :3]  # Remove alpha channel

        # Overlay Grad-CAM on the original image with transparency
        overlay_image = overlay_grad_cam_on_image(image_np, cam_image, alpha=0.5)

        # Add labels to the image
        overlay_image_with_labels = add_labels_to_image(overlay_image, true, pred)

        fig = plot_grad_cam(image_np, true, pred, overlay_image_with_labels)

        # Convert the Matplotlib figure to a Tensor
        fig_tensor = figure_to_tensor(fig)

        # Log the subplot figure to TensorBoard
        writer.add_image(f"Grad-CAM/{log_name}/True_{true}_Pred_{pred}_{idx}", fig_tensor)

        # Close the Matplotlib figure to free memory
        plt.close(fig)
        
       
def overlay_grad_cam_on_image(original_image, cam_image, alpha=0.5, scale_factor=2):
    """Overlays the Grad-CAM image on the original image with some transparency."""
    
    # Apply overlay with transparency (alpha blending)
    overlay = original_image * (1 - alpha) + cam_image * alpha
    overlay = np.clip(overlay, 0, 1)  # Ensure values are valid for display

    # Convert to uint8 for visualization
    overlay_uint8 = (overlay * 255).astype(np.uint8)
    return overlay_uint8

def plot_grad_cam(image, true, pred, overlay_image):
    """Plots the original image and Grad-CAM overlay image side by side."""
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the original image
    ax[0].imshow(image)
    ax[0].set_title(f"Original Image\nTrue: {true}, Pred: {pred}")
    ax[0].axis('off')

    # Plot the Grad-CAM overlay image
    ax[1].imshow(overlay_image)
    ax[1].set_title(f"Grad-CAM Overlay\nTrue: {true}, Pred: {pred}")
    ax[1].axis('off')

    return fig


def figure_to_tensor(fig):
    """Converts a Matplotlib figure to a PyTorch Tensor for TensorBoard."""
    buf = BytesIO()
    fig.savefig(buf, format='png')  # Save the figure to a buffer
    buf.seek(0)
    image = Image.open(buf).convert("RGB")
    tensor = transforms.ToTensor()(image)  # Convert to a tensor
    buf.close()
    return tensor

In [21]:
def log_integrated_gradients_old(model, inputs, labels, device, epoch, writer):
    """
    Applies Integrated Gradients to a single batch of inputs and logs a side-by-side plot (input + IG) to TensorBoard.
    """
    model.eval()
    ig = IntegratedGradients(model)

    inputs = inputs.to(device)
    labels = labels.to(device)

    # Perform IG for a single input (e.g., the first image in the batch)
    input_example = inputs[0].unsqueeze(0)  # Add batch dimension
    target_label = labels[0].item()  # Ground truth class

    # Compute attributions
    attributions = ig.attribute(input_example, target=target_label, n_steps=50)

    # Convert tensors to numpy for visualization
    input_example = input_example.squeeze().cpu().detach().numpy()
    attributions = attributions.squeeze().cpu().detach().numpy()

    # Normalize attributions for visualization
    attributions = (attributions - attributions.min()) / (attributions.max() - attributions.min())

    # Reshape input and attribution for RGB or Grayscale
    input_example = input_example.transpose(1, 2, 0)  # CHW to HWC for RGB
    if input_example.shape[2] != 3:  # Handle grayscale
        input_example = np.repeat(input_example[:, :, np.newaxis], 3, axis=2)
    attribution_overlay = attributions.transpose(1, 2, 0)

    # Create a side-by-side plot
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(input_example)
    axes[0].set_title("Input Image")
    axes[0].axis("off")
    axes[1].imshow(attribution_overlay, cmap="viridis")
    axes[1].set_title("Integrated Gradients")
    axes[1].axis("off")

    # Save the plot to a buffer
    fig.canvas.draw()
    buf = fig.canvas.tostring_rgb()
    width, height = fig.canvas.get_width_height()
    image = np.frombuffer(buf, dtype=np.uint8).reshape(height, width, 3).transpose(2, 0, 1)  # HWC to CHW

    plt.close(fig)  # Close the plot to free memory

    # Log the combined image to TensorBoard
    writer.add_image(f"IG/Input_and_IG", image, epoch)

    print(f"Integrated Gradients side-by-side plot logged for epoch {epoch}")

In [22]:
def denormalize(image, mean, std):
    """
    Denormalizes a tensor image.
    
    Args:
        image (Tensor): Normalized image of shape [C, H, W].
        mean (Tensor): Mean used for normalization.
        std (Tensor): Standard deviation used for normalization.
    
    Returns:
        Tensor: Denormalized image of shape [C, H, W].
    """
    for c in range(image.size(0)):
        image[c] = image[c] * std[c] + mean[c]
    return image

def apply_grad_cam(model, images, preds, true_labels, writer, log_name="Validation"):
    # Assume 'model' is in eval mode and the images are pre-processed
    gradcam = GradCAMPlusPlus(model)
    for i, image in enumerate(images):
        image_tensor = transforms.ToTensor()(image).unsqueeze(0).cuda()
        true_label = true_labels[i]
        
        # Get Grad-CAM++ heatmap
        cam = gradcam(image_tensor)
        cam = cam.squeeze().cpu().detach().numpy()  # Remove batch dimension
        
        # Normalize the CAM output
        cam = np.maximum(cam, 0)  # Set negative values to 0
        cam = cam / cam.max()  # Normalize to [0, 1]

        denormalized_image = denormalize_image(image_tensor.unsqueeze(0)).cpu().numpy().transpose(1, 2, 0)
        
        # Overlay heatmap on the original image
        cam = np.uint8(255 * cam)  # Convert to 0-255 range
        cam = np.expand_dims(cam, axis=-1)  # Make it a 3-channel image
        heatmap = np.repeat(cam, 3, axis=-1)
        overlayed_image = np.uint8(denormalized_image * 0.5 + heatmap * 0.5)  # Blend the original image with heatmap

        # Convert image to TensorBoard format
        figure = plt.figure(figsize=(10, 10))
        plt.subplot(1, 2, 1)
        plt.imshow(denormalized_image)
        plt.title(f"Original - True Label: {true_label}, Pred: {preds[i]}")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(overlayed_image)
        plt.title(f"Grad-CAM++ Overlay")
        plt.axis('off')

        writer.add_figure(f'{log_name}/Grad-CAM++_{i}', figure)

#### TRAINING AND VALIDATION

In [23]:
def train_one_epoch(model, train_loader, criterion, optimizer, scheduler, device, epoch, log_name, writer):
    model.train()  # Set model to training mode
    running_loss = 0.0
    running_corrects = 0

    # Iterate over training data
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch} Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients

        # Forward pass
        outputs = model(inputs)
        outputs = nn.Softmax(dim=1)(outputs)  # Apply softmax for classification
        preds = torch.argmax(outputs, dim = 1)

        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    # Compute loss and accuracy for the epoch
    epoch_loss = running_loss / (len(train_loader) * train_loader.batch_size)
    epoch_acc = running_corrects.double() / (len(train_loader) * train_loader.batch_size)

    # Log statistics
    writer.add_scalar(f'Loss/Train/{log_name}', epoch_loss, epoch)
    writer.add_scalar(f'Accuracy/Train/{log_name}', epoch_acc, epoch)

    # Step the scheduler
    scheduler.step()

    print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
    return epoch_acc, epoch_loss


In [24]:
def validate_one_epoch(model, validation_loader, criterion, device, epoch, num_epochs, log_name, writer):
    model.eval()  # Set model to evaluation mode
    
    running_loss = 0.0
    running_corrects = 0
    all_preds = []
    all_labels = []
    misclassified_images = []
    misclassified_preds = []
    misclassified_true = []

    # Disable gradient computation for validation
    with torch.no_grad():
        for inputs, labels in tqdm(validation_loader, desc=f"Epoch {epoch} Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            outputs = nn.Softmax(dim=1)(outputs)  # Apply softmax for classification
            #_, preds = torch.max(outputs, 1)
            preds = torch.argmax(outputs, dim = 1)
        
            loss = criterion(outputs, labels)

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            # Collect all predictions and labels for confusion matrix
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
         
            # Collect misclassified images
            misclassified = preds != labels.data
            if misclassified.any():
                misclassified_images.extend(inputs[misclassified].cpu())
                misclassified_preds.extend(preds[misclassified].cpu().numpy())
                misclassified_true.extend(labels[misclassified].cpu().numpy())

    # Compute loss and accuracy for the epoch
    epoch_loss = running_loss / (len(validation_loader) * validation_loader.batch_size)
    epoch_acc = running_corrects.double() / (len(validation_loader) * validation_loader.batch_size)

    # Log statistics
    writer.add_scalar(f'Loss/Validation/{log_name}', epoch_loss, epoch)
    writer.add_scalar(f'Accuracy/Validation/{log_name}', epoch_acc, epoch)

    print(f"Validation Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    
    # Confusion Matrix
    log_confusion_matrix(all_labels, all_preds, epoch, log_name, writer)
    
    # Log misclassified images (with confusion matrix info)
    # log_misclassified_images(misclassified_images, misclassified_preds, misclassified_true, epoch, writer)

    # Apply Grad-CAM to a subset of 30 random validation images
    subset_indices = np.random.choice(len(misclassified_images), size=30, replace=False)
    subset_images = [misclassified_images[i] for i in subset_indices]
    subset_preds = [misclassified_preds[i] for i in subset_indices]
    subset_true = [misclassified_true[i] for i in subset_indices]

    # apply_grad_cam(model, subset_images, subset_preds, subset_true, writer, log_name="Validation")


    #Integrated Gradients
    # log_integrated_gradients(model, inputs, labels, device, epoch, writer)


    return epoch_acc, epoch_loss


In [25]:
def fine_tune(model, train_loader, validation_loader, criterion, optimizer, scheduler, early_stop, num_epochs=100, 
              log_name='run1', device='cuda', writer=None):
    best_model = copy.deepcopy(model)
    best_acc = 0.0
    best_epoch = 0
    stop = False

    for epoch in range(1, num_epochs + 1):
        if stop:
            break
        print(f'Epoch {epoch}/{num_epochs}')
        print('-'*120)

        # Training phase
        train_acc, train_loss = train_one_epoch(model, train_loader, criterion, optimizer, scheduler, device, epoch, log_name, writer)

        # Validation phase
        val_acc, val_loss = validate_one_epoch(model, validation_loader, criterion, device, epoch, num_epochs, log_name, writer)

        # Early stopping
        if val_acc > best_acc:
            best_acc = val_acc
            best_epoch = epoch
            best_model = copy.deepcopy(model)

        early_stop(val_loss)
        stop = early_stop.early_stop

        print('-'*120)

    print(f'Best val Acc: {best_acc:4f}')
    print(f'Best epoch: {best_epoch:03d}')
    
    # Save the best model state when training stops
    model_path = os.path.join('saved_models/convnext_model', 'RealArt_vs_FakeArt_convnext_base_augmented.pt')
    torch.save(best_model.state_dict(), model_path)
    print(f"Model saved to: {model_path}")

    return best_model

In [ ]:
model_path = os.path.join('saved_models/convnext_model', 'RealArt_vs_FakeArt_convnext_base.pt')
writer = SummaryWriter("runs/gradcam")

if not os.path.exists(model_path):
   print("Training a new model...")
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.Adam(model.parameters(), lr=1e-3)
   scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
   early_stop = EarlyStopping(patience = 3, min_delta = 0.001)
   ImageFile.LOAD_TRUNCATED_IMAGES = True

   
   # Train and save the best model
   best_model_head = fine_tune(model, train_loader, validation_loader, criterion, optimizer, scheduler, 
                               early_stop, num_epochs = 30, writer = writer)
else: 
     # Model already exists; load it
     print("Loading pre-trained model...")
     model = timm.create_model('convnext_base',pretrained=True, num_classes=2) 
     model.load_state_dict(torch.load(model_path))
     model.to(device)
     print("Model loaded successfully from:", model_path)

Loading pre-trained model...
Model loaded successfully from: saved_models/convnext_model/RealArt_vs_FakeArt_convnext_base.pt


TESTING

In [30]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

# testing function
def test_model(model, test_loader):
    model.eval() # evaluation mode
    test_loss = 0
    correct = 0
    pred_list = []
    true_list = []
    misclassified_images = []
    misclassified_preds = []
    misclassified_true = []


    # progress bar
    pbar = tqdm(total=len(test_loader))
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sommo il loss di ogni batch
            
            pred = output.argmax(dim=1, keepdim=True) # ottengo la predizione del modello
            pred_list.extend(pred.cpu().numpy()) # aggiungo la predizione alla lista
            true_list.extend(target.cpu().numpy()) # aggiungo il target alla lista
            
            correct += pred.eq(target.view_as(pred)).sum().item() # aggiorno il contatore di classificazioni corrette

            misclassified = ~pred.eq(target.view_as(pred)).squeeze()
            if misclassified.any():
                misclassified_images.extend(data[misclassified].cpu())
                misclassified_preds.extend(pred[misclassified].cpu().numpy())
                misclassified_true.extend(target[misclassified].cpu().numpy())

            # update progress bar
            pbar.update(1)
            
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    recall = recall_score(true_list, pred_list, average='macro')
    precision = precision_score(true_list, pred_list, average='macro') 
    f1 = f1_score(true_list, pred_list, average='macro') 
    auc = roc_auc_score(true_list, pred_list) 
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%), Recall: {:.2f}%, Precision: {:.2f}%, F1: {:.2f}%, AUC: {:.2f}%\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy, recall*100, precision*100, f1*100, auc*100))
    
    log_misclassified_images(misclassified_images, misclassified_preds, misclassified_true, None, writer)

    # apply_grad_cam(model, misclassified_images, misclassified_preds, misclassified_true, writer, log_name = "Testing")

    writer.flush()
    return accuracy, recall, precision, f1, auc


100%|██████████| 276/276 [00:40<00:00, 12.20it/s]

use this for logs:
tensorboard --logdir=./runs

In [32]:
accuracy,recall,precision,f1,auc = test_model(model,test_loader)
writer.close()

/tmp/ipykernel_333378/1298213628.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  text = f'True: {int(true_label)}, Pred: {int(pred_label)}'



Test set: Average loss: 0.0752, Accuracy: 8643/8816 (98.04%), Recall: 97.90%, Precision: 98.10%, F1: 98.00%, AUC: 97.90%

Logging misclassified images...
173


100%|██████████| 276/276 [00:25<00:00, 10.63it/s]
